##Datascience Homework 2##
Author: Aedan Wells

Date: 10/03/22

Brief: Program for a user to output and interpret grade information for multiple assignments and rules. This includes applying weights for overall grades, dropping low grades when appropriate, and outputting statistical information.

This entry completes all portions asked:


*   Prints table 1 with adjustable columns to make sure all data fits
*   Prints table 2 using raw values of student grades as data source for it was not specified
*   Prints table 3 using the pearson coefficient values
*   Utilizes only the numpy library when possible and python provided functions
*   All section specifics such as csv name stored as a constant and no crashes was considered
*   Completed extra credit, added drop directive and printing table 4 in a clean manner

Notes:
* Extra credit was completed but it is currently commented out so you can see how the code performs without it. I am not confident that the values after dropping are right so it is commented out
* To see the drop directive and Table 4 for the extra credit, uncomment the lines beneath the comments "UNCOMMENT THE BELOW FOR EC PORTION" within the main function. These should be the calls 'data, table4 = drop_directive(data, about_data)' and 'print_table4(table4)'



In [ ]:
import numpy as np

In [ ]:
#Splits the about data into individual arrays for easier parsing
#param about_data is the about_data array given from the CSV
#returns each row separated with the first label removed
def split_data(about_data):                                                         
  assert (about_data[0,0], about_data[1,0], about_data[2,0], about_data[3,0]) ==  ('HEADERS',
                         'WEIGHTS', 'OUTOF', 'GROUP'), '***Input file ERROR in Row headers'
  names = about_data[0, 1:]
  about = about_data[1:4, 1:]  # keep only the numeric part                                   
  # convert each empty string to nan                                                    
  with np.nditer(about_data, op_flags = ['readwrite']) as row:
    for item in row:
      if item[()] =="":  # get value from scalar ndarray of zero dimension           
        item[...] = np.nan

  about = np.asfarray(about) # convert to float                                        

  weights, outof, group = about[0], about[1], about[2]
  return weights, outof, group, names;

In [ ]:
#Uses the data and about data table to create the table 1 output
#For each student, print the ID, overall score for each group and overall score
#param data is the raw student data (points per assignment)
#param about_data is the csv of assignment descriptors
def compute_student(data, about_data):
  weights, outof, group,headers = split_data(about_data)
  #grab number of groups and info matrix filled with 0's
  group_num = len(np.unique(group))
  info = np.zeros((len(data), group_num))
  student_counter = 0
  #turn nan to 0's for math assistance
  data_nan = np.nan_to_num(data)
  #for each student in the list
  for student in data_nan:
    #assign ID in 0th index
    info[student_counter, 0] = student[0]
    #for each group hw
    for index in range(len(group)):
      group_val = group[index]
      #if it is not group 0 or a nan, do the summing for the table
      if group_val != 0 and not np.isnan(group_val):
        info[student_counter, int(group_val)] = info[student_counter, int(group_val)] + (student[index] * weights[index])/outof[index]
    #make overall grade column value
    for i in range(1, group_num - 1):
      info[student_counter, group_num - 1] = info[student_counter, group_num - 1] + info[student_counter, i]
    student_counter = student_counter + 1
  #print table 1
  info = np.around(info, 2)
  print_table1(info, group_num)
  #print table 2
  table2(data, about_data)
  #print table 3
  table3(info, data)

In [ ]:
#Function to print table1 so there is not too much code in compute_student
#param info is the final info table for the students
def print_table1(info, group_num):
  print("\n")
  print("Table 1:")
  info_T = info.T.astype(str)
  #get sizes for printed -- divider
  size = len(max(info_T[0], key=len))+6
  #print headers for the table, formatted to the size of the greatest element
  print("+{0:^{1}}|".format("ID", size-2), end ="")
  #for each group, print Group #. Modify width for longest value in the list
  for i in range(info[1].size - 2):
    groupname = "Group " + str((i + 1))
    print("{0:^{1}}|".format(groupname, len(max(info_T[i+1], key=len))+4), end ="")
    size = size + len(max(info_T[i+1], key=len))+5
  #pritn overall header
  print("{0:^{1}}+".format("Overall", len(max(info_T[i+1], key=len))+4))
  #print ---- divider
  size = size + len(max(info_T[i+1], key=len))+5
  print("-"*size)
  #for each student, print the data
  for row in info:
    print("+{0:^{1}}|".format(str(row[0]), len(max(info_T[0], key=len))+4), end ="")
    for group in range(info[1].size-2):
      print("{0:^{1}}|".format(str(row[group+1]), len(max(info_T[group+1], key=len))+4), end ="")
    print("{0:^{1}}+".format(str(row[group+2]), len(max(info_T[group+1], key=len))+4))
  print("-"*size)

In [ ]:
#Uses the data and about data table to create the table 2 output
#for each item, print the average, std, max, and min of that class
#param data is the raw student data (points per assignment)
#param about_data is the csv of assignment descriptors
def table2(data, about_data):
  weights, outof, group, headers = split_data(about_data)
  #get transposition of the data table, results in each column being each 
  #students value for a HW
  data_T = data.T

  print("\n")
  print("Table 2:")
  print("------------------------------------------------------------------------")
  print("+ Assignment Name |  Average  |  Standard Dev  |  Maximum  |  Minimum  +")
  print("------------------------------------------------------------------------")
  #for each group
  for i in range(np.shape(data_T)[0] - 1):
    mean = str(round(np.nanmean(data.T[i + 1]),2))
    std = str(round(np.nanstd(data.T[i + 1]),2))
    max = str(round(np.nanmax(data.T[i + 1]),2))
    min = str(round(np.nanmin(data.T[i + 1]),2))
    print('+{:^17s}|{:^11s}|{:^16s}|{:^11s}|{:^11s}+'.format(headers[i+1], mean, std, max, min))
  print("------------------------------------------------------------------------")

In [ ]:
#Uses the data and about data table to create the table 3 output\
#collects and prints the correlation coefficient between attendance, last grade, and ID for overall grade
#param info is the info table that has the overall grade value
#param data is the data table that holds the attendance, ID, and last grade value
def table3(info, data):
  info_T = info.T
  data_T = np.nan_to_num(data).T
  #get coef values
  attend_coef = np.corrcoef(info_T[-1,:], data_T[1])
  lastgrade_coef = np.corrcoef(info_T[-1,:], data_T[2])
  id_coef = np.corrcoef(info_T[-1,:], data_T[0])
  #round values, get right value to print
  attend_coef = np.around(attend_coef, 3)[0,1]
  lastgrade_coef = np.around(lastgrade_coef, 3)[0,1]
  id_coef = np.around(id_coef, 3)[0,1]
  #print values
  print("\n")
  print("Table 3:")
  print("----------------------------------------")
  print("Category    |  Pearson Correlation Value")
  print("----------------------------------------")
  print("Attendance  |  " + str(attend_coef))
  print("LastGrade   |  " + str(lastgrade_coef))
  print("ID          |  " + str(id_coef))
  print("----------------------------------------")

In [ ]:
#completes the extra credit section for performing the drop directive
#param data is the raw student data (points per assignment)
#param about_data is the csv of assignment descriptors
#returns the data array with the dropped value and the table4 array to be printed
def drop_directive(data, about_data):
  #get array with all rows that have the drop directive
  drop = about_data[np.where(about_data[:,0] == "DROP_ONE_OF")]
  header = about_data[np.where(about_data[:,0] == "HEADERS")][0]
  #create what will be table4, add empty section for ignored section
  dropped = np.copy(data)
  dropped = np.concatenate((dropped, np.empty((len(data),len(drop)), dtype="U100")), axis = 1)
  #for each possible directive
  for directive in drop:
    group_to_drop = directive[1]
    #grab indexes for the group being dropped
    group_index = np.where(about_data[3,:] == group_to_drop)
    student_counter = 0
    #for each student, find the smallest value among the drop directive group
    for student in data:
      min = np.infty
      min_index = -1
      for index in group_index[0]:
        if student[index-1] < min:
          min = student[index-1]
          min_index = index-1
      #if min has not changed, must have no assignment in that group
      if min == np.infty:
        dropped[student_counter, np.size(dropped, 1) - 1] = "No Assignment in Drop Group"
      #else, set that value to nan and label what was dropped
      else:
        student[min_index] = np.nan
        dropped[student_counter, np.size(dropped, 1) - 1] = "Item " + header[min_index+1] + " was ignored score" 
      student_counter = student_counter + 1
  return data, dropped

In [ ]:
#prints the table4 in a neat fashion
#param table4 is the table4 data in numpy area fashion
def print_table4(table4):
  print("\n")
  print("Table 4:")
  table4_T = table4.T.astype(str)
  for row in table4:
    print("+{0:^{1}}|".format(str(row[0]), len(max(table4_T[0], key=len))+4), end ="")
    for i in range(table4[0].size - 2):
      print("{0:^{1}}|".format(str(row[i + 1]), len(max(table4_T[i + 1], key=len))+4), end ="")
    print("{0:^{1}}+".format(str(row[i + 2]), len(max(table4_T[i + 1], key=len))+4))

In [ ]:
def main():
  #gather file names
  DATAFILE = input('Enter the filename that contains the data: ')
  ABOUT_DATA = input('Enter the filename that contains the data contexts: ')
  #Read in CSV's
  data = np.genfromtxt(DATAFILE, delimiter = ',')
  about_data = np.genfromtxt(ABOUT_DATA, delimiter = ',',dtype='str')
  #UNCOMMENT THE BELOW FOR EC PORTION
  #data, table4 = drop_directive(data, about_data)
  compute_student(data, about_data)
  #UNCOMMENT THE BELOW FOR EC PORTION
  #print_table4(table4)

In [ ]:
main()

Enter the filename that contains the data: data.csv
Enter the filename that contains the data contexts: about-data.csv


Table 1:
+   ID    | Group 1 |Group 2 | Group 3 | Overall +
--------------------------------------------------
+  100.0  |  35.75  |  17.4  |  36.71  |  89.86  +
+  101.0  |  26.69  |  22.5  |  30.06  |  79.25  +
+  102.0  |  4.89   |  0.0   |  6.93   |  11.82  +
+  103.0  |  20.99  |  11.7  |  14.64  |  47.33  +
+  104.0  |  4.74   |  0.0   |  7.66   |  12.41  +
+  105.0  |  30.92  |  13.2  |  37.12  |  81.24  +
+  106.0  |  25.03  |  15.6  |  21.58  |  62.21  +
+  107.0  |  31.84  |  21.3  |  39.48  |  92.62  +
+  109.0  |  35.09  |  22.2  |  41.2   |  98.49  +
+  110.0  |  30.28  |  12.6  |  11.7   |  54.57  +
+  111.0  |  30.99  |  16.2  |  34.94  |  82.13  +
+  112.0  |  29.32  |  16.5  |  37.8   |  83.62  +
+  113.0  |  25.12  |  16.8  |  31.09  |  73.01  +
+  114.0  |  33.11  |  21.0  |  32.6   |  86.72  +
+  115.0  |   4.6   |  1.2   |  11.02  |  16.82  +
+  